In [1]:
import numpy as np
import slideio
import cv2
import json
import matplotlib.path as mpltPath
import os
import warnings
import tifffile
import staintools
from skimage import exposure
from skimage.exposure import match_histograms
import xml.etree.ElementTree as ET
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt

In [2]:
MMRstatus_data=pd.read_csv('../MMRstatus.csv',encoding='cp949')
P53_data=pd.read_csv('../P53.csv',encoding='cp949')
def svsToTiff(imageCount,magnification,tile_size,slideName):
    global svsWidth,svsHeight,path,scene,fileName
    reduction_rate=360
    slide_tile_size=524
    count=0
    for widthCount in range(0, int(svsWidth // slide_tile_size)):
        for heightCount in range(0, int(svsHeight // slide_tile_size)):
            points = np.array([path.contains_point(np.array([widthCount* slide_tile_size, heightCount* slide_tile_size])), path.contains_point(np.array([widthCount* slide_tile_size, (heightCount+1)* slide_tile_size])), 
                                 path.contains_point(np.array([(widthCount+1)* slide_tile_size, heightCount* slide_tile_size])),
                                path.contains_point(np.array([(widthCount+1)* slide_tile_size, (heightCount+1)* slide_tile_size]))])
            inside = np.mean(points)>0.7
            Full=np.mean(points)>0
            if (inside):
                image = scene.read_block((widthCount * slide_tile_size, heightCount * slide_tile_size, slide_tile_size, slide_tile_size),size=(256,256))
                if np.where(image[:,:,1]>=230)[0].shape[0]<(256*256)/2:
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    TiffImageSave(image, fileName, imageCount,slide_tile_size,slideName)
                    imageCount += 1
                    if(imageCount==4):
                        break;
        if(imageCount==4):
            break;

    return imageCount
def TiffImageSave(image1,fileName,imageCount,slide_tile_size,slideName):
    global normalizer2x,normalizer5x,normalizer10x,tmp_img 
    image2 = normalizer2x.transform(image1)
    cv2.imwrite('../data/256data/input/' + fileName + "_" + str(
        imageCount)+ '.tiff', image1)
    cv2.imwrite('../data/256data/output/' + fileName + "_" + str(
        imageCount)+ '.tiff', image2)
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")    
    

In [3]:
warnings.filterwarnings(action='ignore')
files = os.listdir('../svs_data/')
svs_files=[]
xml_files=[]
for i in range(len(files)):
    svs_class=files[i].find('.svs')
    xml_class=files[i].find('.xml')
    if svs_class!=-1:
        svs_files.append(files[i])
    elif xml_class!=-1:
        xml_files.append(files[i])
tmp_img_2x=cv2.imread('./temp2.5x.tiff')
tmp_img_5x=cv2.imread('./temp5x.tiff')
tmp_img_10x=cv2.imread('./temp10x.tiff')
normalizer2x= staintools.StainNormalizer('macenko')
normalizer2x.fit(tmp_img_2x)
normalizer5x= staintools.StainNormalizer('macenko')
normalizer5x.fit(tmp_img_5x)
normalizer10x= staintools.StainNormalizer('macenko')
normalizer10x.fit(tmp_img_10x)

In [4]:
for i in range(len(svs_files)):
    try:
        file_extension_index=svs_files[i].find('.svs')
        fileName=svs_files[i][:file_extension_index]
        slideName=svs_files[i][:9]
        slide = slideio.open_slide('../svs_data/'+fileName+'.svs', "SVS")
        scene = slide.get_scene(0)
        svsWidth = scene.rect[2]
        svsHeight = scene.rect[3]
        imageCount=0
        magnification=80
        tile_size=256
        xml_open='../svs_data/'+fileName+'.xml'
        polygon_xml = ET.parse(xml_open)
        root = polygon_xml.getroot()
        root = root.getchildren()
        root =root[0].getchildren()
        for annotation_count in range(len(root)):
            root1 =root[annotation_count].getchildren()
            root1 =root1[0].getchildren()
            polygon=[]
            for polygon_count in range(len(root1)): 
                polygon.append([int(float(root1[polygon_count].attrib['X'])),int(float(root1[polygon_count].attrib['Y']))])
            path = mpltPath.Path(polygon)
            imageCount=svsToTiff(imageCount,magnification,tile_size,slideName)
            if(imageCount==4):
                break
    except:
        file_extension_index=svs_files[i].find('.svs')
        fileName=svs_files[i][:file_extension_index]
        print(fileName)

S10-01345-1
S10-06438-2
S10-06438-16
S10-07753-6
S10-27147-4R
S10-27147-5R
S10-28950-7
S10-28950-8
S11-13522-9
S11-14059-1
S11-15498-9
S11-18614-2
S11-23853-MG4
S11-25637-12
S11-25637-15
S11-27049-1
S11-27049-2
S11-27296-2
S11-27296-3
S11-27296-4
S11-27749-15
S11-31465-15
S11-33397-8R
S12-02539-7
S12-05129-1
S12-05129-9
S12-15923-MG8
S12-18293-4
S12-25754-12
S12-26702-1
S12-30332-16
S12-30332-17
S13-16862-7
S13-17955-3
S13-17955-11
S13-17955-12
S13-19937-23
S13-23875-16R
S14-04726-2
S14-04976-5
S14-04976-8
S14-05313-4
S14-05313-5
S14-06111-6
S14-11763-1
S14-11763-2
S14-13997-2
S14-21094-2
S14-21094-3
S14-27557-5
S14-29043-3
S14-29043-4
S15-01797-1
S15-02984-36
S15-03278-7
S15-03278-8
S15-03278-9
S15-03845-8
S15-04670-1
S15-06063-2
